# Learn Keras for Deep Neural Networks
## Chapter 6 - The Path Foward

### MNIST example for Convolutional Neural Networks

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils

/Users/jmoolay/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/jmoolay/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#Loading data from Keras datasets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
#Defining the height and weight and number of samples
#Each Image is a 28 x 28 with 1 channel matrix
training_samples, height, width = x_train.shape
testing_samples,_,_ = x_test.shape

print("Training Samples:",training_samples)
print("Testing Samples:",testing_samples)
print("Height: "+str(height)+" x Width:"+ str(width))

Training Samples: 60000
Testing Samples: 10000
Height: 28 x Width:28


In [ ]:
#Lets have a look at a sample image in the training data
plt.imshow(x_train[0],cmap='gray', interpolation='none')

In [5]:
#We now have to engineer the image data into the right form
#For CNN, we would need the data in Height x Width X Channels form
#Since the image is in grayscale, we will use channel = 1
channel =1
x_train = x_train.reshape(training_samples, height, width,channel).astype('float32')
x_test = x_test.reshape(testing_samples, height, width, channel).astype('float32')

#To improve the training process, we would need to standardizee or normalize the values
#We can achieve this using a simple divide by 256 for all values
x_train = x_train/255
x_test =x_test/255

#Total number of digits  =10
target_classes = 10

# numbers 0-9, so ten classes
n_classes = 10

# convert integer labels into one-hot vectors
y_train = np_utils.to_categorical(y_train, n_classes)
y_test = np_utils.to_categorical(y_test, n_classes)

In [6]:
#Desing the CNN Model
model = Sequential()
model.add(Conv2D(64, (5, 5), input_shape=(height,width ,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(n_classes, activation='softmax'))

In [7]:
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
# Fit the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=200)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 61s 1ms/step - loss: 0.2452 - acc: 0.9266 - val_loss: 0.0627 - val_acc: 0.9806
Epoch 2/10
60000/60000 [==============================] - 64s 1ms/step - loss: 0.0651 - acc: 0.9804 - val_loss: 0.0414 - val_acc: 0.9860
Epoch 3/10
60000/60000 [==============================] - 62s 1ms/step - loss: 0.0457 - acc: 0.9858 - val_loss: 0.0274 - val_acc: 0.9912
Epoch 4/10
60000/60000 [==============================] - 62s 1ms/step - loss: 0.0374 - acc: 0.9885 - val_loss: 0.0288 - val_acc: 0.9907
Epoch 5/10
60000/60000 [==============================] - 61s 1ms/step - loss: 0.0298 - acc: 0.9908 - val_loss: 0.0280 - val_acc: 0.9904
Epoch 6/10
60000/60000 [==============================] - 62s 1ms/step - loss: 0.0259 - acc: 0.9917 - val_loss: 0.0254 - val_acc: 0.9918
Epoch 7/10
60000/60000 [==============================] - 64s 1ms/step - loss: 0.0218 - acc: 0.9930 - val_loss: 0.0224 - val_acc

In [21]:
#Finally let's evaluate the model performance
metrics = model.evaluate(x_test, y_test, verbose=0)
for i in range(0,len(model.metrics_names)):
    print(str(model.metrics_names[i])+" = "+str(metrics[i]))

loss = 0.02039033946258933
acc = 0.9936


### IMDB text classification example for RNN/LSTM

In [2]:
#Import the necessary packages
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

#Setting a max cap for the number of distinct words
top_words = 5000
#Loading the training and test data from keras datasets
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=top_words)

In [4]:
#Since the length of each text will be varying
#We will pad the sequences (i.e. text) to get a uniform length throughout 
max_text_length = 500
x_train = sequence.pad_sequences(x_train, maxlen=max_text_length)
x_test = sequence.pad_sequences(x_test, maxlen=max_text_length)

In [8]:
#Design the network
embedding_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_length, input_length=max_text_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

#Compile the  model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#Fit the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 226s 9ms/step - loss: 0.5312 - acc: 0.7176 - val_loss: 0.5133 - val_acc: 0.7521
Epoch 2/3
25000/25000 [==============================] - 219s 9ms/step - loss: 0.3137 - acc: 0.8710 - val_loss: 0.3116 - val_acc: 0.8737
Epoch 3/3
25000/25000 [==============================] - 223s 9ms/step - loss: 0.2616 - acc: 0.8986 - val_loss: 0.3872 - val_acc: 0.8277


In [26]:
#Evaluate the accuracy on the test dataset
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy:",scores[1])

Accuracy: 0.73584


## Further readling links
Consolidating few reading links and additional references for the readers of the book to explore additional references and other advanced topics 

### Recommeded books for Machine Learning with Python

* Mastering Machine Learning with Python in 6 steps - https://www.apress.com/us/book/9781484228654
* Pactical Machine Learning with Python - https://www.apress.com/us/book/9781484232064


### Recommeded books for Deep Learning with Python/Tensorflow
* Applied Deep Learning -  https://www.apress.com/us/book/9781484237892
* Pro Deep Learning with Tensorflow - https://www.apress.com/us/book/9781484230954



### Reading Links Convolutional Neural Networks (CNN)
* https://adeshpande3.github.io/adeshpande3.github.io/A-Beginner's-Guide-To-Understanding-Convolutional-Neural-Networks/
* https://medium.freecodecamp.org/an-intuitive-guide-to-convolutional-neural-networks-260c2de0a050


### Reading Links for Recurrent Neural Networks
* https://colah.github.io/posts/2015-08-Understanding-LSTMs/
* https://medium.com/mlreview/understanding-lstm-and-its-diagrams-37e2f46f1714
* https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21


### Reading Links for Image Captioning  (CNN + RNN)
* https://towardsdatascience.com/image-captioning-in-deep-learning-9cd23fb4d8d2
* https://machinelearningmastery.com/introduction-neural-machine-translation/
* https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd


### Reading Links for Generative Adversarial Networks 
* https://medium.com/@awjuliani/generative-adversarial-networks-explained-with-a-classic-spongebob-squarepants-episode-54deab2fce39
* https://medium.com/ai-society/gans-from-scratch-1-a-deep-introduction-with-code-in-pytorch-and-tensorflow-cb03cdcdba0f
* https://towardsdatascience.com/understanding-generative-adversarial-networks-4dafc963f2ef